In [2]:
import os
import csv
from time import *

from numpy import mean, std

from greedy_algorithm import greedy_solver
from ilp_model import ilp_solver
from genetic_algorithm import GeneticAlgorithm

from scripts.graph_reader import read_graph
from scripts.graph_plotter import plot_graph

In [3]:
dir_instances = sorted(os.listdir("../instances"))

In [4]:
def solve(include_greedy, include_ilp, include_ga, ga_selection_method, ga_crossover_rate, ga_mutation_rate, ga_logger):
    for subdir in dir_instances:
        dim_path = os.path.join("../instances/", dir_instances)
        instance_dim = ""

        if subdir.endswith("0"):
           instance_dim = "0"
        elif subdir.endswith("1"):
           instance_dim = "1"
        elif subdir.endswith("2"):
            instance_dim = "2"

        with open("../results/results" + instance_dim + ".csv", "w", newline="") as csvfile:
            writer = csv.writer(csvfile)

            # header
            writer.writerow(["Instances", "Greedy algorithm", "ILP", "Genetic algorithm"])
            instance_info = ""

            # prolazak kroz sve instance odredjene dimenzije
            for instance in sorted(os.listdir(os.path.abspath(dim_path))):
                if instance != "imgs":
                    instance_path = os.path.join(dim_path, instance)

                    if dim_path.endswith("0"):
                        instance_info = instance.split(".txt")[0]
                    else:
                        instance_info = instance.split(".clq")[0]

                    vertices_w, edges = read_graph(instance_path, False)

                    num_vertices = len(vertices_w)
                    num_edges = len(edges)


                    if include_greedy:
                        start = time()
                        dominating_set, weight = greedy_solver(vertices_w, edges)
                        end = time()
                        elapsed = end - start
                        greedy_results = "|Ds| = " + str(len(dominating_set)) + "\nweight = " + str(weight) +"\nT(s) = " + f"{elapsed:.3f}"
                        print("Greedy ", instance, " done")
                    else:
                        greedy_results = 0


                    if include_ilp:
                        start = time()
                        dominating_set, weight = ilp_solver(vertices_w, edges)
                        end = time()
                        elapsed = end - start
                        ilp_results = "|Ds| = " + str(len(dominating_set)) + "\nweight = " + str(weight) +"\nT(s) = " + f"{elapsed:.3f}"
                        print("ILP ", instance, " done")
                    else:
                        ilp_results = 0

                    if include_ga:
                        solutions = []
                        times = []
                        for i in range(2):
                            ga = GeneticAlgorithm(vertices_w, edges, population_size=40, chromosome_length=num_vertices, selection_method=ga_selection_method, crossover_rate=ga_crossover_rate, mutation_rate=ga_mutation_rate, n_generations=100, max_time=600, logger=ga_logger)
                            start = time()
                            solution = ga.run()
                            end = time()
                            solutions.append(solution)
                            times.append(end - start)

                        dominating_set = [s.fitness[0] for s in solutions]
                        min_index = dominating_set.index(min(dominating_set))
                        min_wds = len([i for i in solutions[min_index].chromosome if i == 1])
                        ga_results = "|Ds| = " + str(min_wds) + "\nmean = " + str(mean(dominating_set)) + "\nmax = " + str(max(dominating_set)) + "\nmin = " + str(min(dominating_set)) + "\nsd = " + str(round(std(dominating_set), 3)) +"\nmean(T) = " + str(round(mean(times), 3))

                        print("GA ", instance, " done")
                    else:
                        ga_results = 0

                    instance_info += str("\n|V| = ") + str(num_vertices) + "\n|E| = " + str(num_edges)
                    writer.writerow([instance_info, greedy_results, ilp_results, ga_results])


In [1]:
solve(include_greedy=False, include_ilp=False, include_ga=True, ga_selection_method="tournament", ga_crossover_rate=0.8, ga_mutation_rate=0.01, ga_logger=False)